In [ ]:
!pip install netcdf4

In [2]:
!pip install torch==1.4.0

     |████████████████████████████████| 753.4MB 22kB/s 
ERROR: torchvision 0.8.1+cu101 has requirement torch==1.7.0, but you'll have torch 1.4.0 which is incompatible.
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101


In [1]:
import torch
import numpy as np
import xarray as xr
import pandas as pd
import torch.nn as nn
import random
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader

In [ ]:
te = np.load('/content/drive/MyDrive/test_0144-01-12.npy')

In [ ]:
module = SimpleTCNN()

In [ ]:
device = 'cuda'
file = 'test_0144-01-12.npy'
va = torch.from_numpy((np.squeeze(te[:,:,:,3:4]))[np.newaxis,:,:,:]).to(device).float()
ua = torch.from_numpy((np.squeeze(te[:,:,:,2:3]))[np.newaxis,:,:,:]).to(device).float()
t300 = torch.from_numpy((np.squeeze(te[:,:,:,1:2]))[np.newaxis,:,:,:]).to(device).float()
sst = torch.from_numpy((np.squeeze(te[:,:,:,0:1]))[np.newaxis,:,:,:]).to(device).float()
module.load_state_dict(torch.load('/content/drive/MyDrive/toynet.pkl'))
module.eval()
module.to(device)
a = module(sst,t300,ua,va)
#np.save('/content/drive/MyDrive/result/'+file,a.cpu().detach().numpy())
a.cpu().detach().numpy().shape

(1, 24)

In [ ]:
np.squeeze(a.cpu().detach().numpy())

array([ 0.02612521, -0.05007014, -0.03303062, -0.07526279, -0.05865184,
       -0.07897149,  0.00475834, -0.07527208, -0.05578967, -0.00021856,
        0.01431238, -0.04349803,  0.06047166,  0.00518712,  0.02079447,
       -0.06532962,  0.05837119,  0.03829683, -0.03480838, -0.06997815,
       -0.06586695,  0.02465218,  0.07073278, -0.04991773], dtype=float32)

In [ ]:
import zipfile
import os
def make_zip(source_dir='/content/drive/MyDrive/result/', output_filename = 'result.zip'):
    zipf = zipfile.ZipFile(output_filename, 'w')
    pre_len = len(os.path.dirname(source_dir))
    source_dirs = os.walk(source_dir)
    print(source_dirs)
    for parent, dirnames, filenames in source_dirs:
        print(parent, dirnames)
        for filename in filenames:
            if '.npy' not in filename:
                continue
            pathfile = os.path.join(parent, filename)
            arcname = pathfile[pre_len:].strip(os.path.sep)   #相对路径
            zipf.write(pathfile, arcname)
    zipf.close()
make_zip()

<generator object walk at 0x7f89f0d24200>
/content/drive/MyDrive/result/ []


In [3]:
import os
def seed_everything(seed = 427):
    random.seed(seed)
    np.random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
#     torch.set_deterministic(True)
seed_everything()

In [4]:
SODA_train = xr.open_dataset('/content/drive/MyDrive/enso_round1_train_20210201/SODA_train.nc')
SODA_label = xr.open_dataset('/content/drive/MyDrive/enso_round1_train_20210201/SODA_label.nc')

In [14]:
CMIP_train = xr.open_dataset('/content/drive/MyDrive/enso_round1_train_20210201/CMIP_train.nc')
CMIP_label = xr.open_dataset('/content/drive/MyDrive/enso_round1_train_20210201/CMIP_label.nc')

In [ ]:
SODA_label

<xarray.Dataset>
Dimensions:  (month: 36, year: 100)
Coordinates:
  * year     (year) int32 1 2 3 4 5 6 7 8 9 10 ... 92 93 94 95 96 97 98 99 100
  * month    (month) int32 1 2 3 4 5 6 7 8 9 10 ... 28 29 30 31 32 33 34 35 36
Data variables:
    nino     (year, month) float64 ...

In [ ]:
train_label = SODA_label['nino']

In [15]:
class SimpleTCNN(nn.Module):
    def __init__(self, n_cnn_layer:int=1, kernals:list=[3], n_lstm_units:int=64):
        super(SimpleTCNN, self).__init__()
        self.conv1 = nn.ModuleList([nn.Conv2d(in_channels=12, out_channels=12, kernel_size=i) for i in kernals])
        self.conv2 = nn.ModuleList([nn.Conv2d(in_channels=12, out_channels=12, kernel_size=i) for i in kernals])
        self.conv3 = nn.ModuleList([nn.Conv2d(in_channels=12, out_channels=12, kernel_size=i) for i in kernals])
        self.conv4 = nn.ModuleList([nn.Conv2d(in_channels=12, out_channels=12, kernel_size=i) for i in kernals])
        self.max_pool = nn.AdaptiveAvgPool2d((22, 1))
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 128))
        self.batch_norm = nn.BatchNorm1d(12, affine=False)
        self.lstm = nn.LSTM(88, n_lstm_units, 2, bidirectional=True)
        self.linear = nn.Linear(128, 24)
    def forward(self, sst, t300, ua, va):
        for conv1 in self.conv1:
            sst = conv1(sst)
        for conv2 in self.conv2:
            t300 = conv2(t300)
        for conv3 in self.conv3:
            ua = conv3(ua)
        for conv4 in self.conv4:
            va = conv4(va)
        sst = self.max_pool(sst).squeeze(dim=-1)
        t300 = self.max_pool(t300).squeeze(dim=-1)
        ua = self.max_pool(ua).squeeze(dim=-1)
        va = self.max_pool(va).squeeze(dim=-1)
        
        x = torch.cat([sst, t300, ua, va], dim=-1)
        x = self.batch_norm(x)
        x, _ = self.lstm(x)
        x = self.avg_pool(x).squeeze(dim=-2)
        x = self.linear(x)
        return x

In [ ]:
print (torch.__version__)

1.7.0+cu101


In [ ]:
SODA_train['sst'][:,:]

<xarray.DataArray 'sst' (year: 100, month: 36, lat: 24, lon: 72)>
[6220800 values with dtype=float32]
Coordinates:
  * year     (year) int32 1 2 3 4 5 6 7 8 9 10 ... 92 93 94 95 96 97 98 99 100
  * month    (month) int32 1 2 3 4 5 6 7 8 9 10 ... 28 29 30 31 32 33 34 35 36
  * lat      (lat) float64 -55.0 -50.0 -45.0 -40.0 -35.0 ... 45.0 50.0 55.0 60.0
  * lon      (lon) float64 0.0 5.0 10.0 15.0 20.0 ... 340.0 345.0 350.0 355.0

In [7]:
train_label = SODA_label['nino'][:, 12:36].values
train_sst = SODA_train['sst'][:, :12].values
train_t300 = SODA_train['t300'][:, :12].values
train_ua = SODA_train['ua'][:, :12].values
train_va = SODA_train['va'][:, :12].values

In [16]:
train_label = CMIP_label['nino'][:, 12:36].values
train_sst = CMIP_train['sst'][:, :12].values
train_t300 = CMIP_train['t300'][:, :12].values
train_ua = CMIP_train['ua'][:, :12].values
train_va = CMIP_train['va'][:, :12].values

In [ ]:
test_label = SODA_label['nino'][:, 12:36].values
test_sst = SODA_train['sst'][:, :12].values
test_t300 = SODA_train['t300'][:, :12].values
test_ua = SODA_train['ua'][:, :12].values
test_va = SODA_train['va'][:, :12].values

In [17]:
train_ua = np.nan_to_num(train_ua)
train_va = np.nan_to_num(train_va)
train_t300 = np.nan_to_num(train_t300)
train_sst = np.nan_to_num(train_sst)

In [ ]:
train_sst.shape

(100, 12, 24, 72)

In [18]:
train_index, valid_index = train_test_split(range(train_label.shape[0]),test_size=0.2, random_state=427)

In [19]:
class EarthDataSet(Dataset):
    def __init__(self, data):
        self.data = data
    def __len__(self):
        return len(self.data['sst'])
    def __getitem__(self, idx):
        
        return {
            'sst':self.data['sst'][idx],
            't300':self.data['t300'][idx],
            'ua':self.data['ua'][idx],
            'va':self.data['va'][idx],
            'label':self.data['label'][idx]
        }

In [20]:
dict_train = {
    'sst':train_sst[train_index],
    't300':train_t300[train_index],
    'ua':train_ua[train_index],
    'va': train_va[train_index],
    'label': train_label[train_index]}
dict_valid = {
    'sst':train_sst[valid_index],
    't300':train_t300[valid_index],
    'ua':train_ua[valid_index],
    'va': train_va[valid_index],
    'label': train_label[valid_index]}

In [21]:
train_dataset=EarthDataSet(dict_train)
valid_dataset = EarthDataSet(dict_valid)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=True)

In [ ]:
train_dataset.data['sst'][0:1,0:1,:,:]

array([[[[-1.9924349 , -2.4209225 , -1.9242461 , ..., -1.1226535 ,
          -1.146272  , -2.1330566 ],
         [-1.0873079 , -1.3833748 , -0.8232015 , ..., -0.51751316,
          -0.49160358, -0.966813  ],
         [ 0.03181179,  0.43280944,  0.54323304, ..., -0.3987563 ,
          -0.13451098, -0.86001664],
         ...,
         [ 0.00798325, -0.00410122,  0.        , ...,  0.613102  ,
           0.81503975,  1.319501  ],
         [-0.27292472, -0.23630363,  0.2554794 , ...,  0.43132603,
           0.19787297,  0.41840318],
         [-0.02171358,  0.06511507,  0.18634444, ...,  0.3257914 ,
           0.09227768,  0.1151861 ]]]], dtype=float32)

In [22]:
model = SimpleTCNN()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
# device = 'cpu'
model = model.to(device)
optim = torch.optim.Adam(model.parameters(), lr=2e-6)
criterion = nn.L1Loss()

model.to(device)
criterion.to(device)
nums_epoch = 500
for i in range(nums_epoch):
    model.train()
    for data in tqdm(train_loader):
        sst = data['sst'].to(device).float()
        t300 = data['t300'].to(device).float()
        ua = data['ua'].to(device).float()
        va = data['va'].to(device).float()
        optim.zero_grad()
        label = data['label'].to(device).float()
        preds = model(sst, t300, ua, va)
        loss = criterion(preds, label)
        loss.backward()
        optim.step()
    model.eval()
    losses = []
    for data in tqdm(valid_loader):
        sst = data['sst'].to(device).float()
        t300 = data['t300'].to(device).float()
        ua = data['ua'].to(device).float()
        va = data['va'].to(device).float()
        label = data['label'].to(device).float()
        preds = model(sst, t300, ua, va)
        loss = criterion(preds, label)
        losses.append(loss.cpu().detach().numpy())
    print(f'losses: {np.mean(losses)}')
torch.save(model.state_dict(), '/content/drive/MyDrive/cimp.pkl')

  4%|▍         | 9/233 [00:00<00:02, 82.76it/s]

losses: 0.6679771542549133


  3%|▎         | 8/233 [00:00<00:03, 74.84it/s]

losses: 0.6585805416107178


  4%|▍         | 9/233 [00:00<00:02, 81.20it/s]

...

losses: 0.5502935647964478


  3%|▎         | 8/233 [00:00<00:03, 73.53it/s]

losses: 0.540978729724884


100%|██████████| 59/59 [00:00<00:00, 124.05it/s]

losses: 0.5417125225067139


In [ ]:
model.load_state_dict(torch.load(''))